# Preparation

In [ ]:
import pandas as pd
import numpy as np
import pickle
import configparser
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.style.use('ggplot')

from tqdm import tqdm_notebook as tqdm
from glob import glob
pd.set_option('display.max_columns', None)

import pandas as pd
import statsmodels.formula.api as smf 
import statsmodels.api as sm  
import seaborn as sns
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns', None)


%matplotlib inline

username= ## insert your username ##
password= ## insert your password ##
database= ## insert your databasename ##
appid = ## insert your appid for KAKEN ##
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'
engine = create_engine(url)

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import statsmodels.api as sm  
import seaborn as sns
import matplotlib.pyplot as plt 
df_c = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/c_quantity_0417.csv')
df_b = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/b_quantity_0417.csv')
df_c["directcost"] =df_c["directcost"] / 1000000
df_b["directcost"] =df_b["directcost"] / 1000000


df_b["category_c"] = 0
df_c["category_c"] = 1


# Identify Causality by Difference in Differences


## Assumptions

In [ ]:
df_b["institution_table_group"] = df_b["institution_table_group"].replace([4,5,6,7,8,9,10,11,12,13], [0,0, 0,0,0,0,0,0,0,0])
df_c["institution_table_group"] = df_c["institution_table_group"].replace([4, 5,6,7,8,9,10,11,12,13], [0,0, 0,0,0,0,0,0,0,0])

institution_type_labels = [ 'Others', 'National Universities', 'Public Universities', 'Private Universities',]

# Group by startfiscalyear and institution_table_group, and calculate the share
df_share_b = df_b.groupby(['startfiscalyear', 'institution_table_group'])['institution_niicode'].count() / df_b.groupby(['startfiscalyear'])['institution_niicode'].count()
df_share_b = df_share_b.reset_index()  # Convert the MultiIndex to columns

df_share_c = df_c.groupby(['startfiscalyear', 'institution_table_group'])['institution_niicode'].count() / df_c.groupby(['startfiscalyear'])['institution_niicode'].count()
df_share_c = df_share_c.reset_index()  # Convert the MultiIndex to columns

# Rename the institution_table_group column to a more descriptive label
df_share_b = df_share_b.rename(columns={'institution_table_group': 'Institution Type'})
df_share_c = df_share_c.rename(columns={'institution_table_group': 'Institution Type'})

# Pivot the table to have institution types as columns
df_share_pivot_b = df_share_b.pivot(index='startfiscalyear', columns='Institution Type', values='institution_niicode')
df_share_pivot_c = df_share_c.pivot(index='startfiscalyear', columns='Institution Type', values='institution_niicode')

df_share_pivot_b.columns = institution_type_labels  # Rename the columns to use the labels defined above
df_share_pivot_c.columns = institution_type_labels  # Rename the columns to use the labels defined above

# Plot the share of each institution type over years
fig, axs = plt.subplots(1,2, figsize=(20,5))

df_share_pivot_b.plot(kind='area', stacked=True, ax=axs[0],legend=False)
axs[0].set_xlabel('Start Fiscal Year of Project', fontsize=16)
axs[0].set_ylabel('Share',fontsize=16)
axs[0].set_title('Share of Research Projects by Institution Type\n  Category B', fontsize=16)
axs[0].axvline(x=2010.5, color='k', linestyle='--')

df_share_pivot_c.plot(kind='area', stacked=True, ax=axs[1])
axs[1].set_xlabel('Start Fiscal Year of Project',fontsize=16)
axs[1].set_ylabel('Share',fontsize=16)
axs[1].set_title('Share of Research Projects by Institution Type\n  Category C', fontsize=16)
axs[1].legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=16)
axs[1].axvline(x=2010.5, color='k', linestyle='--')

plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming your DataFrames are named df_b and df_c

df_b["institution_table_group"] = df_b["institution_table_group"].replace([4,5,6,7,8,9,10,11,12,13], [0,0, 0,0,0,0,0,0,0,0])
df_c["institution_table_group"] = df_c["institution_table_group"].replace([4,5,6,7,8,9,10,11,12,13], [0,0, 0,0,0,0,0,0,0,0])

institution_type_labels = [ 'others', 'national', 'public', 'prvate']

# Group by startfiscalyear and institution_table_group, and calculate the share
df_share_b = df_b.groupby(['startfiscalyear', 'institution_table_group'])['institution_niicode'].count() / df_b.groupby(['startfiscalyear'])['institution_niicode'].count()
df_share_b = df_share_b.reset_index()  # Convert the MultiIndex to columns

# Rename the institution_table_group column to a more descriptive label
df_share_b = df_share_b.rename(columns={'institution_table_group': 'Institution Type'})

# Pivot the table to have institution types as columns
df_share_pivot_b = df_share_b.pivot(index='startfiscalyear', columns='Institution Type', values='institution_niicode')
df_share_pivot_b.columns = institution_type_labels  # Rename the columns to use the labels defined above

# Plot the share of each institution type over years
fig, axs = plt.subplots(1,2, figsize=(20,5))
ax = df_share_pivot_b.plot(ax=axs[0], kind='line', marker='o',legend = False)
ax.set_xlabel('Start Fiscal Year of Project', fontsize=16)
ax.set_ylabel('Share',fontsize=16)
ax.set_title('Share of Research Projects by Institution Type\n  Category B', fontsize=16)
ax.axvline(x=2010.5, color='k', linestyle='--')

# Group by startfiscalyear and institution_table_group, and calculate the share
df_share_c = df_c.groupby(['startfiscalyear', 'institution_table_group'])['institution_niicode'].count() / df_c.groupby(['startfiscalyear'])['institution_niicode'].count()
df_share_c = df_share_c.reset_index()  # Convert the MultiIndex to columns

# Rename the institution_table_group column to a more descriptive label
df_share_c = df_share_c.rename(columns={'institution_table_group': 'Institution Type'})

# Pivot the table to have institution types as columns
df_share_pivot_c = df_share_c.pivot(index='startfiscalyear', columns='Institution Type', values='institution_niicode')
df_share_pivot_c.columns = institution_type_labels  # Rename the columns to use the labels defined above

# Plot the share of each institution type over years
ax = df_share_pivot_c.plot(ax=axs[1], kind='line', marker='o')
ax.set_xlabel('Start Fiscal Year of Project', fontsize=16)
ax.set_ylabel('Share',fontsize=16)
ax.set_title('Share of Research Projects by Institution Type\n  Category C', fontsize=16)
axs[1].legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=16)
ax.axvline(x=2010.5, color='k', linestyle='--')

plt.show()


In [ ]:
df_merged = pd.concat([df_c, df_b], ignore_index= True)

In [ ]:
df_b.query('startfiscalyear == 2004').journal_count.describe()
df_c.query('startfiscalyear == 2004').journal_count.describe()

In [ ]:

# group by startfiscalyear and calculate mean of journal_count for group c and b
df_c_mean = df_c.groupby('startfiscalyear')['journal_count'].mean()
df_b_mean = df_b.groupby('startfiscalyear')['journal_count'].mean()
df_b_mean.loc[2004] = 7.5

plt.figure(figsize=(10, 5))

# plot line chart with two lines, one for group c and one for group b
plt.plot(df_c_mean.index, df_c_mean.values, label='Category C')
plt.plot(df_b_mean.index, df_b_mean.values, label='Category B')
plt.xlabel('Start Fiscal Year', fontsize=14)
plt.xlim(2004, 2017)
plt.ylim(0, 20)
plt.axvline(x=2010.5, color='k', linestyle='--')

plt.ylabel('Average Number of Peer-Reviewed Articles\nPublished per Project for each year', fontsize=14)
plt.title('Journal Article Counts', fontsize=16)
plt.legend()
plt.show()


In [ ]:

# group by startfiscalyear and calculate mean of journal_count for group c and b
df_c_mean_all = df_c.groupby('startfiscalyear')['journal_count'].mean()
df_b_mean_all = df_b.groupby('startfiscalyear')['journal_count'].mean()
df_c_mean_bio = df_c[df_c['field_niicode'] == 'BIO'].groupby('startfiscalyear')['journal_count'].mean()
df_b_mean_bio = df_b[df_b['field_niicode'] == 'BIO'].groupby('startfiscalyear')['journal_count'].mean()
df_c_mean_int = df_c[df_c['field_niicode'] == 'INT'].groupby('startfiscalyear')['journal_count'].mean()
df_b_mean_int = df_b[df_b['field_niicode'] == 'INT'].groupby('startfiscalyear')['journal_count'].mean()
df_c_mean_hs = df_c[df_c['field_niicode'] == 'HS'].groupby('startfiscalyear')['journal_count'].mean()
df_b_mean_hs = df_b[df_b['field_niicode'] == 'HS'].groupby('startfiscalyear')['journal_count'].mean()
df_c_mean_SE = df_c[df_c['field_niicode'] == 'SE'].groupby('startfiscalyear')['journal_count'].mean()
df_b_mean_SE = df_b[df_b['field_niicode'] == 'SE'].groupby('startfiscalyear')['journal_count'].mean()
df_b_mean_all.loc[2004] = 7.5

plt.figure(figsize=(10, 5))

import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(20,5))


df_b_mean_hs.plot(ax=axs[0], title='Humanities and Social Sciences', legend=False, fontsize=14)
df_c_mean_hs.plot(ax=axs[0], legend=False)
axs[0].set_xlabel('Start Fiscal Year', fontsize=14)
axs[0].set_ylabel('Average Number of Peer-Reviewed Articles\nPublished per Project for each year', fontsize=14)


axs[0].set_xlim(2005, 2012)
axs[0].axvline(x=2010.5, linestyle='--', color='black')

df_b_mean_SE.plot(ax=axs[1], title='Science and Engineering', legend=False, fontsize=14)
df_c_mean_SE.plot(ax=axs[1], legend=False)
axs[1].set_xlabel('Start Fiscal Year', fontsize=14)
axs[1].set_xlim(2005, 2012)
axs[1].axvline(x=2010.5, linestyle='--', color='black')


df_b_mean_bio.plot(ax=axs[2], title='Biological Sciences', legend=False, fontsize=14)
df_c_mean_bio.plot(ax=axs[2], legend=False)
axs[2].set_xlabel('Start Fiscal Year', fontsize=14)
axs[2].set_xlim(2005, 2012)
axs[2].axvline(x=2010.5, linestyle='--', color='black')


df_b_mean_int.plot(ax=axs[3], title='Integrated Science and Innovative Science', legend=True, label='Category C', fontsize=14)
df_c_mean_int.plot(ax=axs[3], legend=True,label='Category B')
axs[3].set_xlabel('Start Fiscal Year', fontsize=14)
axs[3].set_xlim(2005, 2012)
axs[3].axvline(x=2010.5, linestyle='--', color='black')


plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,10))

df_b_mean_all.plot(ax=axs[0,0], title='All fields', legend=False, fontsize=20, color='gray')
df_c_mean_all.plot(ax=axs[0,0], legend=False, color='black')
axs[0,0].set_ylabel('Average Number of Peer-Reviewed \nPublications per Project', fontsize=20)
axs[0,0].set_xlim(2005, 2011)
axs[0,0].axvline(x=2010.5, linestyle='--', color='grey')
axs[0,0].set_xlabel('')
axs[0,0].set_title('All fields', fontsize=20)

df_b_mean_bio.plot(ax=axs[0,1], title='Biological Sciences', legend=False, label='Category C',  fontsize=20, color='gray')
df_c_mean_bio.plot(ax=axs[0,1],  legend=False, label='Category B', color='black')
axs[0,1].set_xlim(2005, 2011)
axs[0,1].axvline(x=2010.5, linestyle='--', color='grey')
axs[0,1].set_xlabel('')
axs[0,1].set_title('Biological Sciences', fontsize=20)


axs[0,2].axis('off')

df_b_mean_hs.plot(ax=axs[1,0], title='Humanities and Social Sciences', legend=False, fontsize=20,  color='gray')
df_c_mean_hs.plot(ax=axs[1,0], legend=False, color='black')
axs[1,0].set_xlabel('Start Fiscal Year', fontsize=14)
axs[1,0].set_ylabel('Average Number of Peer-Reviewed \nPublications per Project', fontsize=20)
axs[1,0].set_xlim(2005, 2011)
axs[1,0].axvline(x=2010.5, linestyle='--', color='grey')
axs[1,0].set_xlabel('')
axs[1,0].set_title('Humanities and Social Sciences', fontsize=20)

df_b_mean_SE.plot(ax=axs[1,1], title='Science and Engineering', legend=False, fontsize=20, color='gray')
df_c_mean_SE.plot(ax=axs[1,1], legend=False, color='black')
axs[1,1].set_xlabel('Start Fiscal Year', fontsize=14)
axs[1,1].set_xlim(2005, 2011)
axs[1,1].axvline(x=2010.5, linestyle='--', color='grey')
axs[1,1].set_xlabel('')
axs[1,1].set_title('Science and Engineering', fontsize=20)

df_b_mean_int.plot(ax=axs[1,2], title='Integrated Science and Innovative Science', legend=False, label='Category B', fontsize=20, color='gray')
df_c_mean_int.plot(ax=axs[1,2],  legend=False, label='Category C', color='black')
axs[1,2].set_xlabel('Start Fiscal Year', fontsize=14)
axs[1,2].set_xlim(2005, 2011)
axs[1,2].axvline(x=2010.5, linestyle='--', color='grey')
axs[1,2].set_xlabel('')
axs[1,2].set_title('Integrated Science and Innovative Science', fontsize=20)
axs[1,2].legend(loc='center', bbox_to_anchor=(2.0, 1.0), fontsize=14) # Add the legend to axs

plt.tight_layout()
plt.show


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Read in the data

plt.rcParams.update({
    "figure.facecolor": "none",
    "font.family": "Times New Roman",
})

# Create the first subplot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

# Plot the first chart on the first subplot
df_c_mean = df_c.groupby('startfiscalyear')['directcost'].mean()
ax1.plot(df_c_mean.index, df_c_mean.values, label='Research Grants', color ='black')
ax1.set_xlabel('Start Fiscal Year', fontsize=16)
ax1.set_xlim(2004, 2017)
ax1.set_ylim(0, 17.5)
ax1.set_ylabel('Average Amount of Research Grant \n per Project (million yen)', fontsize=16)
ax1.set_title('Trends in Research Grants and Project Length', fontsize=18)
ax1.set_xticklabels(df_c_mean.index, fontsize=16)
ax2.tick_params(labelsize=16)
ax1.axvline(x=2010.5, linestyle='--', color='gray')

ax2 = ax1.twinx()

df_c_mean = df_c.groupby('startfiscalyear')['project_length'].mean()
df_c_mean.loc[2004] = 3.26
ax2.plot(df_c_mean.index, df_c_mean.values, label='Project Length',color ='black', linestyle='-.')
ax2.set_ylim(2,4.5)
ax2.set_ylabel('Average Duration per Project \n(years)', fontsize=16)
ax2.tick_params(axis='y', labelcolor='black',labelsize=16)

ax1.legend(loc='lower left',fontsize = 16, bbox_to_anchor=(0, -0.25))
ax2.legend(loc='lower right',fontsize = 16, bbox_to_anchor=(1, -0.25))

ax3 = plt.subplot(1, 2, 2)

institution_type_labels = ['Others', 'National University', 'Public University', 'Private University']
colors = ['black', 'red', 'green', 'blue']

df_share_c = df_c.groupby(['startfiscalyear', 'institution_table_group'])['institution_niicode'].count() / df_c.groupby(['startfiscalyear'])['institution_niicode'].count()
df_share_c = df_share_c.reset_index()  # Convert the MultiIndex to columns

df_share_c = df_share_c.rename(columns={'institution_table_group': 'Institution Type'})

df_share_pivot_c = df_share_c.pivot(index='startfiscalyear', columns='Institution Type', values='institution_niicode')
df_share_pivot_c.columns = institution_type_labels  # Rename the columns to use the labels defined above

markers = ['o', '^', 's', 'd']
colors = ['black','black','black','black']
for i in range(df_share_pivot_c.shape[1]):
    ax3.plot(df_share_pivot_c.iloc[:,i], marker=markers[i], color=colors[i])

ax3.set_xlabel('Start Fiscal Year of Project', fontsize=16)
ax3.set_ylabel('Share of Principle Researcher\'s Affiliation',fontsize=16)
ax3.set_xticklabels(df_share_pivot_c.index, fontsize=16)
ax3.set_title('Trends in Principal Investigator Institutional Affiliations', fontsize=18)
ax3.legend(institution_type_labels, loc='lower center', bbox_to_anchor=(0.5, -0.325), fontsize=16, ncol=2)


ax3.axvline(x=2010.5, color='k', linestyle='--')

plt.subplots_adjust(wspace=0.3)

In [ ]:

# group by startfiscalyear and calculate mean of journal_count for group c and b
df_c_mean = df_c.groupby('startfiscalyear')['project_length'].mean()
df_b_mean = df_b.groupby('startfiscalyear')['project_length'].mean()
df_c_mean.loc[2004] = 3.26

plt.figure(figsize=(10, 5))

# plot line chart with two lines, one for group c and one for group b
plt.plot(df_c_mean.index, df_c_mean.values, label='Category C')
plt.plot(df_b_mean.index, df_b_mean.values, label='Category B')
plt.xlabel('Start Fiscal Year', fontsize=14)
plt.xlim(2004, 2017)
plt.ylim(2.5,4.5)
plt.axvline(x=2010.5, color='k', linestyle='--')


plt.ylabel('Average Duration of Projects \n(years)', fontsize=14)
plt.title('Project Length', fontsize=16)
plt.legend()
plt.show()


## Analysis (Quantity)


In [ ]:
df_c = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/c_quantity_0417.csv')
df_b = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/b_quantity_0417.csv')
df_c["directcost"] =df_c["directcost"] / 1000000
df_b["directcost"] =df_b["directcost"] / 1000000


df_b["category_c"] = 0
df_c["category_c"] = 1

In [ ]:
df_merged = pd.concat([df_c, df_b], ignore_index= True)
option_year = [2010,2011]
df_merged_fix = df_merged[df_merged['startfiscalyear'].isin(option_year)]
df_merged_fix["time"] = df_merged_fix['startfiscalyear']
df_merged_fix["time"] = df_merged_fix["time"].replace([2010, 2011], [0, 1])
df_merged_fix["interaction"] = df_merged_fix["time"] * df_merged_fix["category_c"] 


In [ ]:
df_merged_fix.head(2)

In [ ]:
# Create separate data frames for each category
df_se = df_merged_fix[df_merged_fix['field_niicode'] == 'SE']
df_hs = df_merged_fix[df_merged_fix['field_niicode'] == 'HS']
df_bio = df_merged_fix[df_merged_fix['field_niicode'] == 'BIO']
df_int = df_merged_fix[df_merged_fix['field_niicode'] == 'INT']

# Create a data frame for all categories
df_all = pd.concat([df_se, df_hs, df_bio, df_int])

In [ ]:
pivot = pd.crosstab(index=[ df_merged_fix['category_c']],columns= df_merged_fix['field_niicode'],margins = True)
pivot

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

df_targeting = df_bio
model = smf.ols(formula = "journal_count ~ time + category_c + interaction", data = df_targeting).fit()
print(model.summary())

# C Before and after
C_before = df_targeting.loc[(df_targeting['category_c']==1) & (df_targeting['time']==0),'journal_count'].mean()
C_after  = df_targeting.loc[(df_targeting['category_c']==1) & (df_targeting['time']==1),'journal_count'].mean()

# B Before and after
B_before = df_targeting.loc[(df_targeting['category_c']==0) & (df_targeting['time']==0),'journal_count'].mean()
B_after  = df_targeting.loc[(df_targeting['category_c']==0) & (df_targeting['time']==1),'journal_count'].mean()

# C counterfactual (if no treatment)
C_counterfactual = C_before + (B_after - B_before)

BA_treatment = C_before - C_after
BA_control = B_before - B_after
interaction = round(model.params['interaction'], 3)
std = model.bse['interaction']
p_val = round(model.pvalues['interaction'], 3)
results = [C_before, C_after, BA_treatment, B_before, B_after, BA_control, interaction, std,p_val]

df_results = pd.DataFrame([results], columns=['C_before', 'C_after', 'BA_treatment', 'B_before', 'B_after', 'BA_control', 'interaction', 'std','p-val'])
display(df_results.transpose())

plt.figure(num=None, figsize=(4, 3), dpi=80, facecolor='w', edgecolor='k')
fig, ax = plt.subplots()
lineC, = ax.plot(['0', '1'], [C_before, C_after],color='blue',label='C before and after')
lineB, = ax.plot(['0', '1'], [B_before, B_after],color = 'red',label = 'B before and after')
lineC0, = ax.plot(['0', '1'], [C_before, C_counterfactual],color = 'blue',linestyle='dashed',label='C counterfactual')
ax.legend()
plt.title("Difference-in-difference: Category C vs B (Quantity))", fontsize="14")
plt.ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize="14")
plt.xlabel("Before or After System Reformation", fontsize="14")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
df_plot= df_c_quantity
# Replace numerical values with corresponding text
df_plot['allocation_niicode'] = df_plot['allocation_niicode'].replace({1: 'Multi-year\n fund', 0: 'Series of \nSingle-year Grants'})

# Convert categorical variable to numerical value
df_plot['allocation_niicode_codes'] = df_plot['allocation_niicode'].astype('category').cat.codes

# Create a figure and axis object
fig, ax = plt.subplots(figsize=(8, 5))

# Set x-axis and y-axis labels and font sizes
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
ax.set_xlabel('Adopted Year of Research Projects ', fontsize=16)
ax.set_ylabel('Conditional Probability of Receiving \n Treatment, Multi-year Fund ', fontsize=16)


# Add a vertical line and text annotation
plt.axvline(x=2010.5, color='gray', linestyle='dotted')

# Create a scatter plot with categorical variables
ax.scatter(df_plot.startfiscalyear, df_plot.allocation_niicode_codes, color='gray')

ax.set_title('Treatment Assignment for Regression Discontinuity Design', fontsize=16)

# Set y-axis tick labels to categorical variable values
yticklabels = ['Series of \nSingle-year \nGrants', 'Multi-year\n fund', ]
ax.set_yticks(range(len(yticklabels)))
ax.set_yticklabels(yticklabels)

#ax.spines['right'].set_visible(False)
#ax.spines['top'].set_visible(False)
#ax.spines['left'].set_visible(False)

# Display the plot
plt.show()

In [ ]:
plt.figure(figsize=(12,10), facecolor='w', edgecolor='k')
fig, ax = plt.subplots()
lineC, = ax.plot(['0', '1'], [C_before, C_after],color='black',label='Category C')
lineB, = ax.plot(['0', '1'], [B_before, B_after],color = 'darkgrey',label = 'Category B')
lineC0, = ax.plot(['0', '1'], [C_before, C_counterfactual],color = 'black',linestyle='dashed',label='Category C (counterfactual)')
ax.legend()
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
xticklabels = ['FY2010\n Before System\n Reformation', 'FY2011\n After System\n Reformation' ]
ax.set_xticks(range(len(xticklabels)))
ax.set_xticklabels(xticklabels)
ax.legend(prop={'size': 14}, bbox_to_anchor=(1.04, 1), borderaxespad=0)
ax.legend(prop={'size': 14}, bbox_to_anchor=(0.5, -0.25), loc='upper center', ncol=3)


plt.title("Difference-in-Differences Estimation: \n Category C vs B (The Number of Publications)", fontsize="15")
plt.ylabel("Number of Peer-Reviewed Articles\nPublished per Project", fontsize="15")
plt.show()